# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load
file_to_load = "output_data/cities_output.csv"

# Read cities File
city_data = pd.read_csv(file_to_load)
city_data_df= pd.DataFrame(city_data)
city_data_df=city_data_df.rename(columns={"Unnamed: 0":"Id"})
city_data_df


,Id,City,Country,temperature,humidity,cloudiness,wind_speed,latitude,longtitude
0,0,ostrovnoy,RU,0.96,95.0,97.0,6.86,68.0531,39.5131
1,1,mahebourg,MU,25.16,73.0,40.0,7.20,-20.4081,57.7000
2,2,qaanaaq,GL,-20.91,76.0,33.0,3.64,77.4840,-69.3632
3,3,zhuhai,CN,27.65,89.0,40.0,3.09,22.2769,113.5678
4,4,cape town,ZA,11.34,79.0,20.0,1.54,-33.9258,18.4232
...,...,...,...,...,...,...,...,...,...
495,495,fuxin,CN,21.80,17.0,100.0,2.92,42.0156,121.6589
496,496,chernaya kholunitsa,RU,7.95,73.0,100.0,1.87,58.8511,51.7154
497,497,havelock,US,17.45,86.0,4.0,2.53,34.8791,-76.9013
498,498,traverse city,US,19.99,57.0,100.0,7.25,44.7631,-85.6206


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = city_data_df[["latitude", "longtitude"]]

# Fill NaN values and convert to float
humidity = city_data_df["humidity"].astype(float)

Images from the heatmap show high humidity nearer the equator and lower desert regions like the Sahara

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer using max value of humidity ie 100%
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
ideal_cities_df=pd.DataFrame(city_data.loc[(city_data["temperature"]<27) &  
                         (city_data["temperature"]>21) &
                         (city_data["wind_speed"]<15) &
                         (city_data["cloudiness"]==0), :])
ideal_cities_df=ideal_cities_df.rename(columns={"Unnamed: 0": "city_id"})
ideal_cities_df.count() # we have 9 cities in the sample no null values
ideal_cities_df.to_csv('output_data/ideal_cities.csv')
ideal_cities_df

                       

,city_id,City,Country,temperature,humidity,cloudiness,wind_speed,latitude,longtitude
91,91,bella vista,US,22.11,63.0,0.0,9.26,36.4295,-94.2316
118,118,sao filipe,CV,22.98,74.0,0.0,2.60,14.8961,-24.4956
206,206,namioka,JP,23.75,17.0,0.0,7.20,40.7072,140.5894
233,233,san juan del sur,NI,26.09,78.0,0.0,5.60,11.2529,-85.8705
270,270,griffith,US,21.17,58.0,0.0,7.20,41.5284,-87.4237
308,308,nioro,GM,25.90,32.0,0.0,1.41,13.3500,-15.7500
316,316,gunjur,GM,22.04,60.0,0.0,2.06,13.2019,-16.7339
377,377,muskegon,US,21.95,49.0,0.0,9.77,43.3000,-86.1334
473,473,seoul,KR,23.41,56.0,0.0,4.63,37.5683,126.9778


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
# Create a map using state centroid coordinates to set markers

hotel_df=pd.DataFrame(ideal_cities_df)
#hotel_df.insert(2,'Hotel Name','')
city_locations = ideal_cities_df[['latitude', 'longtitude']]

# Create a marker_layer using the ideal cities list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(city_locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))

In [30]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotels",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longtitude"]
    city={row["City"]}
    #print(f"Checking city {city} at index {index}")
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    try:
      name_address = name_address.json()
      hotel_name=name_address["results"][0]["name"]
      hotel_df.loc[index, "Hotel Name"]=name_address["results"][0]["name"]
    #print ('Index is ' + str(index))
    #print(f"In city {city} found an hotel within 5km  {hotel_name}")

    #hotel_df["Hotel Name"].append=hotel_name
    #hotel_df=hotel_df.append({'Hotel Name': hotel_name}, ignore_index=True)
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since cities may not have a hotel within 5 km so
    #  we incorporate a try-except to skip any that are missing a data point.
    except: # (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, "Hotel Name"]="No hotel within 5 km"
    
    

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [31]:
hotel_df.count()
hotel_df.head(20)
hotel_df.to_csv('output_data/hotels.csv')

In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#I changed the line below as I prefer the whole word latitude and longtitude becuase 
#that was how I did it in the Part 1
#locations = hotel_df[["Lat", "Lng"]]
locations = hotel_df[["latitude", "longtitude"]]

In [33]:
# Add marker layer ontop of heat map
# Plot Heatmap
fig = gmaps.figure()
humidity = hotel_df["humidity"].astype(float)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))